In [17]:
import pandas as pd
import numpy as np

from dsgd.DSClassifierMultiQ import DSClassifierMultiQ

from sklearn.metrics import accuracy_score, confusion_matrix, f1_score


In [18]:
df = pd.read_csv("https://huggingface.co/datasets/furrutiav/sac_nllf/raw/main/train.csv")
df = df.drop(columns=['index'])
print("Dataframe shape: ", df.shape)
df.head()
test = pd.read_csv("https://huggingface.co/datasets/furrutiav/sac_nllf/raw/main/val.csv")
test = test.drop(columns=['index'])
print("Dataframe shape: ", test.shape)

Dataframe shape:  (1400, 27)
Dataframe shape:  (200, 27)


In [19]:
y = df['label']
X = df.drop(columns=['label'])
y = y.to_numpy()
X = X.to_numpy()    
y_test = test['label']
X_test = test.drop(columns=['label'])
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

In [20]:
DSCs = {}
time = {}
accuracys = {}
learning_rates = [0.0001, 0.001, 0.01, 0.1]
batch_sizes = [2000, 3000, 4000]
min_dl = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
num_rules = [3, 5, 7,]
for lr in learning_rates:
    for bs in batch_sizes:
        for mdl in min_dl:
            for nr in num_rules:
                DSC = DSClassifierMultiQ(2, min_iter=20, max_iter=500, debug_mode=True, lr=lr, batch_size=bs,
                                lossfn="MSE", num_workers=1, min_dloss=mdl, precompute_rules=True)
                print(f"Training DSC with lr={lr}, bs={bs}, mdl={mdl}, nr={nr}")
                losses, epoch, dt = DSC.fit(X, y, add_single_rules=True,
                            single_rules_breaks=nr, add_mult_rules=False,
                                column_names=df.columns[:-1], print_every_epochs=1, print_final_model=False)
                y_pred = DSC.predict(X_test)
                acc = accuracy_score(y_test, y_pred)
                f1_micro = f1_score(y_test, y_pred, average='micro')
                f1_macro = f1_score(y_test, y_pred, average='macro')
                print(f"Accuracy: {acc}")
                print(f"F1 Micro: {f1_micro}")
                print(f"F1 Macro: {f1_macro}")
                print(confusion_matrix(y_test, y_pred))

Training DSC with lr=0.0001, bs=2000, mdl=1e-07, nr=3
Optimization started
Processing epoch	1	1.0000	

Processing epoch	500	0.2261	
Training time: 1035.46s, epochs: 500

Least training loss reached: 0.226
Accuracy: 0.66
F1 Micro: 0.66
F1 Macro: 0.6591478696741855
[[71 41]
 [27 61]]
Training DSC with lr=0.0001, bs=2000, mdl=1e-07, nr=5
Optimization started
Processing epoch	500	0.2203	
Training time: 1022.46s, epochs: 500

Least training loss reached: 0.220
Accuracy: 0.675
F1 Micro: 0.675
F1 Macro: 0.6689669221562986
[[81 31]
 [34 54]]
Training DSC with lr=0.0001, bs=2000, mdl=1e-07, nr=7
Optimization started
Processing epoch	500	0.2125	
Training time: 1049.96s, epochs: 500

Least training loss reached: 0.212
Accuracy: 0.68
F1 Micro: 0.68
F1 Macro: 0.677386833350136
[[77 35]
 [29 59]]
Training DSC with lr=0.0001, bs=2000, mdl=1e-06, nr=3
Optimization started
Processing epoch	500	0.2159	
Training time: 1054.21s, epochs: 500

Least training loss reached: 0.216
Accuracy: 0.695
F1 Micro: 0.695
F1 Macro: 0.6909123153707786
[[81 31]
 [30 58]]
Training DSC with lr=0.0001, bs=2000, mdl=1e-06, nr=

KeyboardInterrupt: 